## Dataset Download

In [ ]:
!gsutil -m cp -r gs://covidx-bucket/covidx-cxr2/ ./

## Model Variables

In [1]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
DATASET_SIZE = 1500
MAX_EPOCHS = 20
K_FOLDS = 5

## Data Preparation

In [2]:
# import required libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from datetime import datetime
from packaging import version
%reload_ext tensorboard
import tensorboard
from sklearn.model_selection import StratifiedKFold
from tensorboard.plugins.hparams import api as hp
import time

print("Tensorflow Version:", tf.__version__)
print("Tensorboard Version:", tensorboard.__version__)
print("GPU Found:", tf.test.is_gpu_available())

Tensorflow Version: 1.15.5
Tensorboard Version: 1.15.0
GPU Found: True


In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete(['N/A']))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_L_RATE= hp.HParam('learning_rate', hp.Discrete([0.0005, 0.001]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('trained-classifiers/logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_L_RATE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [3]:
# read in train data
train_df = pd.read_csv('covidx-cxr2/train_COVIDx9B.txt', 
                       sep=" ", header=None)
# add columns to go from 0, 1, 2, 3 to patient id, filename, class etc
train_df.columns=['patient id', 'filename', 'class', 'data source']
# drop patient id and datasource as not needed
train_df=train_df.drop(['patient id', 'data source'], axis=1 )

# read in test data
test_df = pd.read_csv('covidx-cxr2/test_COVIDx9B.txt', 
                      sep=" ", header=None)
# add columns to go from 0, 1, 2, 3 to patient id, filename, class etc
test_df.columns=['patient id', 'filename', 'class', 'data source']
# drop patient id and datasource as not needed
test_df=test_df.drop(['patient id', 'data source'], axis=1 )

In [4]:
print("Train class counts:")
print(train_df['class'].value_counts())
print("\nTest class counts:")
print(test_df['class'].value_counts())

Train class counts:
COVID-19     16490
normal        8085
pneumonia     5555
Name: class, dtype: int64

Test class counts:
COVID-19     200
normal       100
pneumonia    100
Name: class, dtype: int64


In [5]:
negative  = train_df[train_df['class']=='negative']   # normal values in class column
positive = train_df[train_df['class']=='positive']  # COVID-19 values in class column

from sklearn.utils import resample
# downsample training data to 400 values of each class, to reduce class bias and reduce training time

df_negative_downsampled = resample(negative, replace = True, n_samples = DATASET_SIZE//2)
df_positive_downsampled = resample(positive, replace = True, n_samples = DATASET_SIZE//2) 

#concatenate
train_df = pd.concat([df_negative_downsampled, df_positive_downsampled])

from sklearn.utils import shuffle
train_df = shuffle(train_df) # shuffling so that there is particular sequence
print("Train class counts:")
print(train_df['class'].value_counts())

Train class counts:
normal       533
pneumonia    533
COVID-19     533
Name: class, dtype: int64


In [6]:
print("Train class counts:")
print(train_df['class'].value_counts())
print("\nTest class counts:")
print(test_df['class'].value_counts())

Train class counts:
normal       483
COVID-19     481
pneumonia    475
Name: class, dtype: int64

Valid class counts:
pneumonia    58
COVID-19     52
normal       50
Name: class, dtype: int64

Test class counts:
COVID-19     200
normal       100
pneumonia    100
Name: class, dtype: int64


In [7]:
# preprocess images
test_datagen = ImageDataGenerator()

test_gen = test_datagen.flow_from_dataframe(dataframe = test_df, directory="covidx-cxr2/test", x_col='filename', 
                                            y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                            color_mode='rgb', class_mode='binary', shuffle=False)

Found 1439 validated image filenames belonging to 3 classes.
Found 160 validated image filenames belonging to 3 classes.
Found 400 validated image filenames belonging to 3 classes.


In [ ]:
def preprocess_images_cv(train_index, val_index):
    training_data = train_df.iloc[train_index]
    validation_data = train_df.iloc[val_index]
    train_datagen = ImageDataGenerator(rotation_range = 20, width_shift_range = 0.2, height_shift_range = 0.2, 
                                       shear_range = 0.2, zoom_range = 0.1, horizontal_flip = True)

    #Now fit the them to get the images from directory (name of the images are given in dataframe) with augmentation

    train_gen = train_datagen.flow_from_dataframe(dataframe = training_data, directory="covidx-cxr2/train", x_col='filename', 
                                                  y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                                  color_mode='rgb', class_mode='binary')
    valid_gen = test_datagen.flow_from_dataframe(dataframe = validation_data, directory="covidx-cxr2/train", x_col='filename',
                                                 y_col='class', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, 
                                                 color_mode='rgb', class_mode='binary')
    return train_gen, valid_gen


## SqueezeNet

In [8]:
# required libraries for ResNet
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [9]:
def fire(x, squeeze, expand):
    y  = keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = keras.layers.BatchNormalization(momentum=0.9)(y)
    y1 = keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = keras.layers.BatchNormalization(momentum=0.9)(y1)
    y3 = keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = keras.layers.BatchNormalization(momentum=0.9)(y3)
    return keras.layers.concatenate([y1, y3])

In [10]:
def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

In [11]:
def create_model(hparams):
    x = keras.layers.Input(shape=[IMAGE_SIZE, IMAGE_SIZE, 3]) 

    y = keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
    y = keras.layers.BatchNormalization(momentum=0.9)(y)
    y = fire_module(24, 48)(y)
    y = keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(64, 128)(y)
    y = keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(24, 48)(y)
    y = keras.layers.GlobalAveragePooling2D()(y)
    # Add dropout to reduce overfitting
    y = keras.layers.Dropout(hparams[HP_DROPOUT])(y)
    y = keras.layers.Dense(1, activation='softmax')(y)

    model = tf.keras.Model(x, y)

    optimizer_name = hparams[HP_OPTIMIZER]
    learning_rate = hparams[HP_L_RATE]
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        raise ValueError("unexpected optimizer name: %r" % (optimizer_name,))


    model.compile(optimizer = optimizer,
                  loss = 'binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def get_model_name(k):
    return str("SqueezeNet-Binary-" + "Fold-"+str(k)+"-"+datetime.now().strftime("%Y%m%d-%H%M%S"))

In [12]:
# perform HP tuning and CV on the model on the data
Y = train_df['class']
trained_models = []

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for learning_rate in HP_L_RATE.domain.values:
                
                skf = StratifiedKFold(n_splits = K_FOLDS, shuffle = True) 
                k_i = 0
                k_fold_models = []
                elapsed_times = []
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
                    HP_L_RATE: learning_rate
                }
                print('HPARAMS:',{h.name: hparams[h] for h in hparams})
                
                for train_index, val_index in skf.split(np.zeros(len(Y)),Y):
                    print('--- Starting trial: %s' % k_i)
                    t = time.process_time()
                    train_gen, valid_gen = preprocess_images_cv(train_index, val_index)

                    logdir = os.path.join('trained classifiers','logs','fit', get_model_name(k_i))
                    hpdir = os.path.join('trained classifiers','logs','hparam_tuning', get_model_name(k_i))
                    modeldir = os.path.join('trained classifiers',str(get_model_name(k_i)+".h5"))
                    callbacks = [
                        keras.callbacks.ModelCheckpoint(modeldir, save_best_only=True, verbose = 0),
                        keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1, restore_best_weights=True),
                        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1),
                        keras.callbacks.TensorBoard(log_dir=logdir)
                      ]
                   
                    hp.hparams(hparams)  # record the values used in this trial
                    model = create_model(hparams)
                    model.fit(train_gen, validation_data=valid_gen, epochs=MAX_EPOCHS, callbacks=callbacks)
                    elapsed_times.append(time.process_time() - t)
                    k_fold_models.append(modeldir)
                    
                    del model, train_gen, valid_gen
                    keras.backend.clear_session()
                    k_i += 1
                    
                trained_models.append([hparams, k_fold_models, elapsed_times])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 200, 200, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 24) 792         batch_normalization[0][0]        
_____________

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from statistics import mean

In [14]:
for j in trained_models:
    model = create_model(j[0])
    accuracies = []
    k_accuracies = []
    precisions = []
    recalls = []
    f1s = []
    tp = []
    fp = []
    tn = []
    fn = []
    for i in j[1]:
        model.load_weights(i)
        test_pred = model.predict(test_gen)
        y_pred = np.rint(test_pred).flatten()
        y_test= [test_gen.class_indices[k] for k in test_df['class'].values.tolist()]
        k_accuracies.append(model.evaluate(test_gen)[1])
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, zero_division=0))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        tn_x, fp_x, fn_x, tp_x = confusion_matrix(y_test, y_pred).ravel()
        tp.append(tp_x)
        fp.append(fp_x)
        tn.append(tn_x)
        fn.append(fn_x)

    print("SCORE FOR HPARAMS:", {h.name: j[0][h] for h in j[0]})
    print("Mean accuracy:", mean(accuracies))
    print("Mean k_accuracy;", mean(k_accuracies))
    print("Mean precision:", mean(precisions))
    print("Mean recall:", mean(recalls))
    print("Mean f1:", mean(f1s))
    print("Mean training time:", mean(j[2]))
    hdict = {h.name: j[0][h] for h in j[0]}
    df =  pd.DataFrame(np.array([[str(j[1]),
                                  hdict.get('num_units'), hdict.get('dropout'), hdict.get('optimizer'), 
                                  hdict.get('learning_rate'),IMAGE_SIZE, BATCH_SIZE, DATASET_SIZE, MAX_EPOCHS, K_FOLDS,
                                  mean(accuracies), mean(k_accuracies), mean(precisions), mean(recalls), mean(f1s),
                                  mean(tn), mean(fp), mean(fn), mean(tp), mean(j[2])
                                 ]]),
                       columns=['Model Names','num_units', 'dropout','optimizer','learning_rate',
                                'IMAGE_SIZE', 'BATCH_SIZE', 'DATASET_SIZE', 'MAX_EPOCHS',
                                'K_FOLDS','Accuracy', 'K_accuracy', 'Precision', 'Recall', 'f1 Score', 
                                'True Negatives', ' False Positives', 'False Negatives', 'True Positives', 'Training Time'])
    with open('trained-classifiers/Squeezenet.csv', 'a') as f:
        df.to_csv(f, mode='a', header=f.tell()==0, index = False)

50/50 [==============================] - 21s 422ms/step - loss: 0.4077 - acc: 0.8725

EVALUATION ON TRAIN DATA
Train loss: 0.6255580113114707
Train acc: 0.7553857

EVALUATION ON VALIDATION DATA
Val loss: 0.7708669796586036
Val acc: 0.65000004

EVALUATION ON TEST DATA
Test loss: 0.40767769783735275
Test acc: 0.8725


NOTE: Accuracy and loss graphs can be seen on TensorBoard, run "%tensorboard --logdir logs --host 0.0.0.0" to open

## Save Weights and Results to Bucket

In [ ]:
!gsutil -m cp -r trained-classifiers gs://trained-classifiers/

In [ ]:
!gsutil -m cp -r model-checkpoints gs://trained-classifiers/